In [1]:
import sys
import os

# Add the parent directory (Finetuning/) to sys.path to access utils_ft
sys.path.append('..')

from utils_ft import (
    get_model_path, 
    get_output_dir, 
    get_processed_data_path, 
    get_adapters_path, 
    to_relative,
    setup_env
)
from scripts.train_mlx import run_mlx_training, fuse_model

# 1. Setup Environment (Load .env variables if needed)
setup_env()

# 2. Config
MODEL_TYPE = "qwen" 
FRAMEWORK = "mlx"

# 3. Define Paths using the new Utils
# This guarantees they match the structure used by the data converter
base_model = get_model_path(MODEL_TYPE)
data_folder = get_processed_data_path()                   # .../Finetuning/data
adapter_path = get_adapters_path(MODEL_TYPE, FRAMEWORK)   # .../Finetuning/adapters/qwen_mlx
fused_output = get_output_dir(MODEL_TYPE, FRAMEWORK)      # .../local_models/ft_qwen_mlx

# 4. Verify Paths (Optional, prints relative paths for clarity)
print(f"--- Configuration ---")
print(f"Base Model:   {base_model}")
print(f"Data Folder:  {to_relative(data_folder)}")
print(f"Adapter Path: {to_relative(adapter_path)}")
print(f"Output Path:  {to_relative(fused_output)}")

--- Configuration ---
Base Model:   Qwen/Qwen2.5-7B-Instruct
Data Folder:  Finetuning/data
Adapter Path: Finetuning/adapters/qwen_mlx
Output Path:  local_models/ft_qwen_mlx


In [2]:
run_mlx_training(base_model, data_folder, adapter_path, iters=900, num_layers=16)

--- Initializing MLX Fine-tuning (CLI Mode) ---
--- Model: ./Qwen/Qwen2.5-7B-Instruct ---
--- Data Folder: ./../data ---
Generating configuration file at: ./../adapters/qwen_mlx/train_config.yaml
Executing MLX CLI training...
Loading configuration file ../adapters/qwen_mlx/train_config.yaml
Loading pretrained model


Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 19716.81it/s]


Loading datasets
Training
Trainable parameters: 0.606% (46.137M/7615.617M)
Starting training..., iters: 900


Calculating loss...: 166it [02:52,  1.04s/it]


Iter 1: Val loss 1.130, Val took 172.328s
Iter 20: Train loss 1.887, Learning Rate 9.992e-06, It/sec 0.493, Tokens/sec 19.884, Trained Tokens 807, Peak mem 18.259 GB
Iter 40: Train loss 0.718, Learning Rate 9.961e-06, It/sec 0.537, Tokens/sec 20.068, Trained Tokens 1555, Peak mem 18.259 GB
Iter 60: Train loss 0.573, Learning Rate 9.905e-06, It/sec 0.483, Tokens/sec 23.745, Trained Tokens 2539, Peak mem 18.580 GB
Iter 80: Train loss 0.374, Learning Rate 9.825e-06, It/sec 0.448, Tokens/sec 20.314, Trained Tokens 3446, Peak mem 18.580 GB


...

Iter 700: Val loss 0.247, Val took 172.472s
Iter 700: Train loss 0.201, Learning Rate 1.215e-06, It/sec 0.483, Tokens/sec 15.480, Trained Tokens 33106, Peak mem 20.142 GB
Iter 700: Saved adapter weights to ../adapters/qwen_mlx/adapters.safetensors and ../adapters/qwen_mlx/0000700_adapters.safetensors.
Iter 720: Train loss 0.298, Learning Rate 9.963e-07, It/sec 0.569, Tokens/sec 23.201, Trained Tokens 33921, Peak mem 20.142 GB
Iter 740: Train loss 0.156, Learning Rate 7.972e-07, It/sec 0.505, Tokens/sec 19.419, Trained Tokens 34690, Peak mem 20.142 GB
Iter 760: Train loss 0.223, Learning Rate 6.185e-07, It/sec 0.448, Tokens/sec 21.483, Trained Tokens 35650, Peak mem 20.142 GB
Iter 780: Train loss 0.277, Learning Rate 4.611e-07, It/sec 0.395, Tokens/sec 33.973, Trained Tokens 37372, Peak mem 20.142 GB


Calculating loss...: 166it [02:52,  1.04s/it]


Iter 800: Val loss 0.247, Val took 172.274s
Iter 800: Train loss 0.200, Learning Rate 3.259e-07, It/sec 0.544, Tokens/sec 15.354, Trained Tokens 37937, Peak mem 20.142 GB
Iter 800: Saved adapter weights to ../adapters/qwen_mlx/adapters.safetensors and ../adapters/qwen_mlx/0000800_adapters.safetensors.
Iter 820: Train loss 0.209, Learning Rate 2.134e-07, It/sec 0.506, Tokens/sec 29.729, Trained Tokens 39111, Peak mem 20.142 GB
Iter 840: Train loss 0.196, Learning Rate 1.243e-07, It/sec 0.540, Tokens/sec 22.330, Trained Tokens 39938, Peak mem 20.142 GB
Iter 860: Train loss 0.191, Learning Rate 5.886e-08, It/sec 0.430, Tokens/sec 20.160, Trained Tokens 40876, Peak mem 20.142 GB
Iter 880: Train loss 0.237, Learning Rate 1.754e-08, It/sec 0.465, Tokens/sec 19.865, Trained Tokens 41730, Peak mem 20.142 GB


Calculating loss...: 166it [02:52,  1.04s/it]


Iter 900: Val loss 0.247, Val took 172.465s
Iter 900: Train loss 0.259, Learning Rate 4.870e-10, It/sec 0.434, Tokens/sec 20.754, Trained Tokens 42686, Peak mem 20.142 GB
Iter 900: Saved adapter weights to ../adapters/qwen_mlx/adapters.safetensors and ../adapters/qwen_mlx/0000900_adapters.safetensors.
Saved final weights to ../adapters/qwen_mlx/adapters.safetensors.
✓ Training completed.


In [3]:
fuse_model(base_model, adapter_path, fused_output)
print(f"Success! Model ready at ./{to_relative(fused_output)}")

Fusing LoRA adapters (Step: Final)...
--- Staging checkpoint from: adapters.safetensors ---
Loading pretrained model


Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 13997.98it/s]


Success! Fused model saved to: ./local_models/ft_qwen_mlx
Success! Model ready at ./local_models/ft_qwen_mlx


In [4]:
# 2. Config
MODEL_TYPE = "llama" 
FRAMEWORK = "mlx"

# 3. Define Paths using the new Utils
# This guarantees they match the structure used by the data converter
base_model = get_model_path(MODEL_TYPE)
data_folder = get_processed_data_path()                   # .../Finetuning/data
adapter_path = get_adapters_path(MODEL_TYPE, FRAMEWORK)   # .../Finetuning/adapters/qwen_mlx
fused_output = get_output_dir(MODEL_TYPE, FRAMEWORK)      # .../local_models/ft_qwen_mlx

# 4. Verify Paths (Optional, prints relative paths for clarity)
print(f"--- Configuration ---")
print(f"Base Model:   {base_model}")
print(f"Data Folder:  {to_relative(data_folder)}")
print(f"Adapter Path: {to_relative(adapter_path)}")
print(f"Output Path:  {to_relative(fused_output)}")

--- Configuration ---
Base Model:   meta-llama/Meta-Llama-3-8B-Instruct
Data Folder:  Finetuning/data
Adapter Path: Finetuning/adapters/llama_mlx
Output Path:  local_models/ft_llama_mlx


In [5]:
run_mlx_training(base_model, data_folder, adapter_path, iters=900, num_layers=16)

--- Initializing MLX Fine-tuning (CLI Mode) ---
--- Model: ./meta-llama/Meta-Llama-3-8B-Instruct ---
--- Data Folder: ./../data ---
Generating configuration file at: ./../adapters/llama_mlx/train_config.yaml
Executing MLX CLI training...
Loading configuration file ../adapters/llama_mlx/train_config.yaml
Loading pretrained model


Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 12179.87it/s]


Loading datasets
Training
Trainable parameters: 0.131% (10.486M/8030.261M)
Starting training..., iters: 900


Calculating loss...: 166it [02:49,  1.02s/it]


Iter 1: Val loss 1.660, Val took 169.101s
Iter 20: Train loss 1.688, Learning Rate 9.992e-06, It/sec 0.530, Tokens/sec 20.553, Trained Tokens 775, Peak mem 18.094 GB
Iter 40: Train loss 0.574, Learning Rate 9.961e-06, It/sec 0.592, Tokens/sec 21.279, Trained Tokens 1494, Peak mem 18.094 GB
Iter 60: Train loss 0.590, Learning Rate 9.905e-06, It/sec 0.529, Tokens/sec 24.588, Trained Tokens 2423, Peak mem 18.233 GB
Iter 80: Train loss 0.353, Learning Rate 9.825e-06, It/sec 0.480, Tokens/sec 21.180, Trained Tokens 3305, Peak mem 18.341 GB


...

Iter 700: Val loss 0.318, Val took 168.868s
Iter 700: Train loss 0.232, Learning Rate 1.215e-06, It/sec 0.531, Tokens/sec 16.308, Trained Tokens 32026, Peak mem 19.506 GB
Iter 700: Saved adapter weights to ../adapters/llama_mlx/adapters.safetensors and ../adapters/llama_mlx/0000700_adapters.safetensors.
Iter 720: Train loss 0.282, Learning Rate 9.963e-07, It/sec 0.625, Tokens/sec 24.781, Trained Tokens 32819, Peak mem 19.506 GB
Iter 740: Train loss 0.165, Learning Rate 7.972e-07, It/sec 0.556, Tokens/sec 20.514, Trained Tokens 33557, Peak mem 19.506 GB
Iter 760: Train loss 0.189, Learning Rate 6.185e-07, It/sec 0.489, Tokens/sec 22.657, Trained Tokens 34484, Peak mem 19.506 GB
Iter 780: Train loss 0.298, Learning Rate 4.611e-07, It/sec 0.419, Tokens/sec 35.351, Trained Tokens 36173, Peak mem 19.506 GB


Calculating loss...: 166it [02:48,  1.02s/it]


Iter 800: Val loss 0.318, Val took 168.891s
Iter 800: Train loss 0.216, Learning Rate 3.259e-07, It/sec 0.597, Tokens/sec 16.186, Trained Tokens 36715, Peak mem 19.506 GB
Iter 800: Saved adapter weights to ../adapters/llama_mlx/adapters.safetensors and ../adapters/llama_mlx/0000800_adapters.safetensors.
Iter 820: Train loss 0.208, Learning Rate 2.134e-07, It/sec 0.552, Tokens/sec 30.560, Trained Tokens 37822, Peak mem 19.506 GB
Iter 840: Train loss 0.146, Learning Rate 1.243e-07, It/sec 0.584, Tokens/sec 23.104, Trained Tokens 38613, Peak mem 19.506 GB
Iter 860: Train loss 0.185, Learning Rate 5.886e-08, It/sec 0.461, Tokens/sec 20.916, Trained Tokens 39521, Peak mem 19.506 GB
Iter 880: Train loss 0.212, Learning Rate 1.754e-08, It/sec 0.500, Tokens/sec 20.799, Trained Tokens 40353, Peak mem 19.506 GB


Calculating loss...: 166it [02:48,  1.02s/it]


Iter 900: Val loss 0.318, Val took 168.943s
Iter 900: Train loss 0.290, Learning Rate 4.870e-10, It/sec 0.466, Tokens/sec 21.722, Trained Tokens 41286, Peak mem 19.506 GB
Iter 900: Saved adapter weights to ../adapters/llama_mlx/adapters.safetensors and ../adapters/llama_mlx/0000900_adapters.safetensors.
Saved final weights to ../adapters/llama_mlx/adapters.safetensors.
✓ Training completed.


In [6]:
fuse_model(base_model, adapter_path, fused_output, 300)
print(f"Success! Model ready at ./{to_relative(fused_output)}")

Fusing LoRA adapters (Step: 300)...
--- Staging checkpoint from: 0000300_adapters.safetensors ---
Loading pretrained model


Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 19574.61it/s]


Success! Fused model saved to: ./local_models/ft_llama_mlx
Success! Model ready at ./local_models/ft_llama_mlx
